In [14]:
import pandas as pd
import numpy as np

import time
import copy

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from graph_tree_module import node, sim, graph_tree




In [15]:
raw = pd.read_csv(r'../Data/train.csv')
# print(raw)

random_seed = 42

# print(list(raw.columns)[:])

In [16]:
data = raw.iloc[:,2:-1]
data = data.fillna(0)
data = data[:5000]
print(data.shape)
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(data)

# label
y = raw.iloc[:,-1]
# print(y)

col_name = list(raw.columns)[2:-1]
print(col_name)
# print(normalized_data)

(5000, 14)
['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_in min/ms', 'time_signature']


In [17]:
data_min = scaler.data_min_
data_max = scaler.data_max_

print(data_min, data_max)

[ 0.0000e+00  5.9900e-02  3.0100e-03  0.0000e+00 -3.6214e+01  0.0000e+00
  2.2500e-02  0.0000e+00  0.0000e+00  1.3600e-02  2.5900e-02  3.4132e+01
  5.0165e-01  1.0000e+00] [9.900000e+01 9.820000e-01 9.980000e-01 1.100000e+01 1.355000e+00
 1.000000e+00 9.370000e-01 9.960000e-01 9.960000e-01 9.920000e-01
 9.850000e-01 2.160910e+02 1.412451e+06 5.000000e+00]


In [18]:
test_tree = graph_tree(attr_name = col_name, X = normalized_data, X_maxmin=[scaler.data_max_,scaler.data_min_], min_cluster_size=200)
# test_tree.show_root()
test_tree.build_tree()

root node is created
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ] [[<graph_tree_module.node object at 0x00000150103247A0>, <graph_tree_module.node object at 0x00000150103247A0>], [<graph_tree_module.node object at 0x00000150103247A0>, <graph_tree_module.node object at 0x00000150103247A0>], [<graph_tree_module.node object at 0x00000150103247A0>, <graph_tree_module.node object at 0x00000150103247A0>], [<graph_tree_module.node object at 0x00000150103247A0>, <graph_tree_module.node object at 0x00000150103247A0>], [<graph_tree_module.node object at 0x00000150103247A0>, <graph_tree_module.node object at 0x00000150103247A0>], [<graph_tree_module.node object at 0x00000150103247A0>, <graph_tree_module.node object at 0x00000150103247A0>], 

0

In [19]:
import pickle

with open("graph_tree.pkl", 'wb') as f:
    pickle.dump(test_tree, f)

In [20]:
test_tree.root.down_lv[0].center

array([0.44017957, 0.51780136, 0.65391656, 0.44026177, 0.75100433,
       1.        , 0.05753974, 0.25911434, 0.13463644, 0.18605759,
       0.47602233, 0.48493261, 0.13730071, 0.72769953])

In [21]:
seaching = np.mean(normalized_data[100:101], axis=0)
print(seaching)
cur = test_tree.search(seaching)

[0.24242424 0.46751979 0.5035126  0.36363636 0.60243818 0.
 0.03728814 0.62751004 0.79718876 0.21300082 0.12417892 0.69124913
 0.18320677 0.75      ]


In [22]:
def super_cluster(cur):
    # super cluster
    print(f"super cluster")
    if cur.up_lv:
        print(cur.up_lv.id)
    else:
        print(f"there is no super cluster")
    print("\n", end="")

def neibour_cluster(cur):
    # neibour in the same lv
    print(f"neibour in the same lv")
    if cur.up_lv:
        if len(cur.up_lv.down_lv) > 1:
            for neibour in cur.up_lv.down_lv:
                if neibour != cur:
                    print(neibour.id)
        else:
            print(f"there is no neibour")
    else:
        print(f"there is no neibour")
    print("\n", end="")

def sub_cluster(cur):
    # sub cluster
    print(f"sub cluster")
    if cur.down_lv:
        for sub in cur.down_lv:
            print(sub.id)
    else:
        print(f"there is no sub cluster")
    print("\n", end="")


def suround_cluster(cur):
    # cur cluster
    print(f"current cluster")
    print(cur.id)
    print("\n", end="")
    
    super_cluster(cur)
    neibour_cluster(cur)
    sub_cluster(cur)
    

In [23]:
# specific searching
# cur = cur.down_lv[0]

#generic searching
# cur = cur.up_lv

#simular searching
# cur = cur.up_lv.down_lv[1]

suround_cluster(cur)

current cluster
15

super cluster
6

neibour in the same lv
16

sub cluster
there is no sub cluster



In [24]:
print(normalized_data.shape)
print(np.mean(normalized_data[:100], axis=0))

print(test_tree.root.center)

(5000, 14)
[0.42919192 0.53548422 0.6287219  0.45818182 0.74335117 0.72
 0.05481575 0.24029304 0.13223035 0.16829518 0.49376186 0.47735809
 0.13475585 0.72      ]
[0.43906465 0.52589225 0.66144496 0.47741818 0.7539839  0.639
 0.06208811 0.25001819 0.13598254 0.1839972  0.47853415 0.48415781
 0.14202648 0.7291    ]


In [25]:
from collections import Counter

"""
create a new index (something like tf-idf)
data reduction rate
vs
accurate
"""
def score(search_data, acc, smoothing = 0.001, lam = 0.5):
    return lam*(acc/100)+(1-lam)*(1-(search_data/(1000+smoothing)))

my_score = []
normal_kmeans_score = []

tmp = [i*100 for i in range(10)]
for i in tmp:
    b = i
    u = i+100
    # print(normalized_data[b:u])
    print(f"-- mine result{i}")
    seaching = np.mean(normalized_data[b:u], axis=0)
    result = test_tree.search(seaching)
    
    print("search number: ", len(result.index))
    right = np.where((result.index>=b) & (result.index<u))[0] # return index
    print("hitted right result:", len(right))
    my_score.append(score(len(result.index), len(right)))

    ############
    print("-- kmeans result")
    checking = KMeans(n_clusters=10, random_state=0).fit(normalized_data)

    counts1 = Counter(checking.labels_)
    print(counts1)

    counts2 = Counter(checking.labels_[b:u])
    print(counts2)
    belongs_group = counts2.most_common()[0][0]
    print("belongs to group: ", belongs_group, "\n")
    # print(counts1.get(belongs_group), counts2.get(belongs_group))
    normal_kmeans_score.append(score(counts1.get(belongs_group), counts2.get(belongs_group)))

    

-- mine result0
search number:  5000
hitted right result: 100
-- kmeans result
Counter({np.int32(9): 944, np.int32(0): 659, np.int32(3): 656, np.int32(1): 619, np.int32(4): 607, np.int32(7): 577, np.int32(5): 299, np.int32(8): 281, np.int32(6): 191, np.int32(2): 167})
Counter({np.int32(9): 22, np.int32(1): 17, np.int32(0): 13, np.int32(4): 12, np.int32(7): 10, np.int32(3): 9, np.int32(8): 6, np.int32(5): 5, np.int32(2): 5, np.int32(6): 1})
belongs to group:  9 

-- mine result100
search number:  5000
hitted right result: 100
-- kmeans result
Counter({np.int32(9): 944, np.int32(0): 659, np.int32(3): 656, np.int32(1): 619, np.int32(4): 607, np.int32(7): 577, np.int32(5): 299, np.int32(8): 281, np.int32(6): 191, np.int32(2): 167})
Counter({np.int32(3): 17, np.int32(9): 15, np.int32(4): 15, np.int32(7): 11, np.int32(5): 10, np.int32(1): 10, np.int32(0): 9, np.int32(6): 6, np.int32(8): 5, np.int32(2): 2})
belongs to group:  3 

-- mine result200
search number:  5000
hitted right result: 100

In [26]:
print(my_score)
print(normal_kmeans_score)

print(sum(my_score))
print(sum(normal_kmeans_score))

[-1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999, -1.4999975000024999]
[0.13800047199952797, 0.25700032799967204, 0.13300047199952797, 0.13300047199952797, 0.27550032949967046, 0.12300047199952799, 0.2965002884997115, 0.28650030349969646, 0.12300047199952799, 0.14300047199952798]
-14.999975000025
1.9085040814959182
